In [1]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import time
import numpy as np
from concurrent.futures import as_completed
from requests_futures.sessions import FuturesSession

In [3]:
#list to hold data
linksl = []
#headers for bot
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
}
#request page
url = "https://www.metacritic.com/browse/games/score/metascore/all/all/filtered"
page = requests.get(url,headers=headers).text

In [ ]:
soup = BeautifulSoup(page,"html.parser")

In [ ]:
table = soup.find('table',{'class':'clamp-list'})
links = table.find_all_next('td',{"class":"clamp-summary-wrap"})

In [ ]:
for i in links:
    linksl.append('https://www.metacritic.com'+i.find_next('a',{'class':'title'})['href'])

In [ ]:
#get pages
pages = []
with FuturesSession() as session:
    futures = [session.get('https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?page='+str(i),headers=headers) for i in range(1,190,1)]
    for future in as_completed(futures):
        pages.append(future.result().content)

In [ ]:
for i in pages:
    soup = BeautifulSoup(i,"html.parser")
    table = soup.find('table',{'class':'clamp-list'})
    links = table.find_all_next('td',{"class":"clamp-summary-wrap"})
    for i in links:
        linksl.append('https://www.metacritic.com'+i.find_next('a',{'class':'title'})['href'])

In [5]:
temp = pd.DataFrame()
df = temp.assign(links = linksl)

In [ ]:
df.to_csv('meta_links.csv',encoding='utf-8',index=False)